In [1]:
#!/usr/bin/python3

from __future__ import unicode_literals, print_function
from pathlib import Path
import pandas as pd
import numpy as np
import random
import spacy
import os
import re

TRAIN_DATA = []

In [2]:
def find_operator(x):
    match = re.search(r'[@:|?()\.,#{}¡!\w]*(:?'+ x["empresas"] +
                      '|'+ x["empresas"].lower() +
                      '|'+ x["empresas"].upper() + ')(?:\w+)?', x["texto"])
    
    operator = np.nan
    if match is not None:
        operator = match.group(0)
    
    return operator
    
def fetch_clean_data():
    files = os.listdir("csv/tweets_raw/")
    df = pd.DataFrame()
    for file in files:
        print("FILE:", file)
        df_file = pd.read_csv("csv/tweets_raw/" + file, sep = "|")
        if len(df) > 0:
            df = df.append(df_file, ignore_index=True)
        else:
            df = df_file   
    
    df["emp_extracted"] = ""
    df["empresas"] = df["empresas"].fillna('')
    empresas = df["empresas"].unique()
    for empresa in empresas:
        df.loc[df["emp_extracted"] == "", "emp_extracted"] = df["texto"].str.extract('(?P<emp_extracted>' +
                                                                           str(empresa) +
                                                                           '|' + str(empresa).upper() +
                                                                           '|' + str(empresa).lower() +')', expand = True)
        df["emp_extracted"] = df["emp_extracted"].fillna("")
    
    df.loc[df["empresas"] == "", "empresas"] = np.nan
    df["conteo_retweets"] = df["conteo_retweets"].values.astype(int)
    df.loc[df["conteo_retweets"] <= 0, "conteo_retweets"] = 0
    df = df.drop(columns=["fecha_creacion", "palabras_clave", "autor", "ciudades"])
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df[df["texto"].str.find("�") == -1]
    df = df.sort_values(by = "tipo_lugar")
    df = df.sort_values(by = "conteo_retweets")
    df = df.drop_duplicates(["texto"], keep="last")
    df.loc[pd.isna(df["empresas"]), "empresas"] = "X°X°X°X°X"
    df["empresas"] = df[["empresas", "texto"]].apply(find_operator, axis = 1)
    df["horas"] = df["texto"].str.extract(
        r'(?P<horas>\d+(?=.{0}[ ]{0,5}?[hH][oO][rR][aA][sS]?))', 
        expand = True
    )
    df["horas"] = df["horas"].fillna(0)
    df["horas"] = df["horas"].astype(int)
    df["dias"] = df["texto"].str.extract(
        r'(?P<dias>\d+(?=.{0}[ ]{0,5}?[dD][iíIÍ][aA][sS]?))', 
        expand = True
    )
    df["dias"] = df["dias"].fillna(0)
    df["dias"] = df["dias"].astype(int)
    df.loc[pd.isna(df["empresas"]), "empresas"] = df["emp_extracted"]
    df = df.drop(columns=["emp_extracted"])
    
    print("CLEANED DATA", len(df), "TWEETS")
    return df

In [3]:
def create_tranning_data():
    df_tweets = fetch_clean_data()
    df_tweets["lugar_start"] = df_tweets[~pd.isna(df_tweets["lugar"])].apply(
        lambda x: int(x["texto"].find(x["lugar"])), 
        axis = 1
    )
    df_tweets["lugar_end"] = df_tweets["lugar_start"] + df_tweets["lugar"].str.len()
    df_tweets["empresa_start"] = df_tweets[~pd.isna(df_tweets["empresas"])].apply(
        lambda x: int(x["texto"].find(x["empresas"])), 
        axis=1
    )
    df_tweets["empresa_end"] = df_tweets["empresa_start"] + df_tweets["empresas"].str.len()
    df_tweets["tipo_lugar_start"] = df_tweets[~pd.isna(df_tweets["tipo_lugar"])].apply(
        lambda x: int(x["texto"].find(x["tipo_lugar"])), 
        axis=1
    )
    df_tweets["tipo_lugar_end"] = df_tweets["tipo_lugar_start"] + df_tweets["tipo_lugar"].str.len()
    
    df_tweets.loc[df_tweets["empresas"] == "", "empresas"] = np.nan
    df_tweets = df_tweets[~pd.isna(df_tweets["empresas"])]
    df_tweets["tipo_lugar"] = df_tweets["tipo_lugar"].fillna("None")
    df_tweets["lugar"] = df_tweets["lugar"].fillna("None")
    
    tweets_lugares = df_tweets.to_dict('records')
    
    for tweet in tweets_lugares:
        if (tweet["lugar"] != "None") and (tweet["tipo_lugar"] != "None"):
            #print(tweet, 'LUGAR Y TIPO LUGAR')
            TRAIN_DATA.append((
                tweet["texto"], {
                    'entities': [
                        (int(tweet["lugar_start"]), int(tweet["lugar_end"]), 'LOC'),
                        (int(tweet["empresa_start"]), int(tweet["empresa_end"]), 'EMP')
                    ],
                    'cats':{tweet["tipo_lugar"].upper(): 0.9}
                }
            ))
        elif (tweet["lugar"] != "None") and (tweet["tipo_lugar"] == "None"):
            #print(tweet, 'SOLO LUGAR')
            TRAIN_DATA.append((
                tweet["texto"], {
                    'entities': [
                        (int(tweet["lugar_start"]), int(tweet["lugar_end"]), 'LOC'),
                        (int(tweet["empresa_start"]), int(tweet["empresa_end"]), 'EMP')
                    ],
                    'cats':{'COLOMBIA':1.0}
                }
            ))
        else:
            #print(tweet, 'NINGUNO')
            TRAIN_DATA.append((
                tweet["texto"], {
                    'entities': [
                        (int(tweet["empresa_start"]), int(tweet["empresa_end"]), 'EMP')
                    ],
                    'cats':{'COLOMBIA':1.0}
                }
            ))
    
    print("CREATED TRAINING DATA", len(TRAIN_DATA), "TWEETS")
    return TRAIN_DATA

In [4]:
def train_model(model="roxanne_model/", output_dir="roxanne_model", n_iter=10):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # Carga un modelo existente
        print("Carga modelo '%s'" % model)
    else:
        nlp = spacy.blank('es')  # Crea un modelo vacio
        print("Crear modelo vacio 'en' model")

    # Crea linea de accion para reconocimiento de labels y entidades
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # Si existe, la obtiene para agregar los labels
    else:
        ner = nlp.get_pipe('ner')

    # Agrega los labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # Excluye otras lineas de accion para solo usar NER
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # Unicamente entrene NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # Texto
                    [annotations],  # Anotaciones
                    drop=0.5,  # Ignora la mitad para dificultar el aprendizaje
                    sgd=optimizer,  # Actualiza los pesos
                    losses=losses)
            print(losses)
            
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)


In [5]:
if __name__ == '__main__':
    create_tranning_data()
    train_model()

FILE: registro_continuo.csv
FILE: tweets_entrenamiento.csv
FILE: filtered_tweets.csv
FILE: tweets_entrenamiento_2.csv
CLEANED DATA 2366 TWEETS
CREATED TRAINING DATA 1657 TWEETS
Carga modelo 'roxanne_model/'
{'ner': 669.2271837170861}


KeyboardInterrupt: 

In [ ]:
def test_model(model="roxanne_model/"):
    # Guarda el modelo en el directorio especificado
    # Prueba el modelo creado
    print("Loading from", model)
    nlp2 = spacy.load(model)
    for text, _ in TRAIN_DATA:
        doc = nlp2(text)
        print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
        print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [111]:
nlp2 = spacy.load("roxanne_model/")
TRAIN_DATA = ["sin luz en el sector de Chinga Tu Madre, @Electricaribe algún daño reportado? gracias"]

for text in TRAIN_DATA:
    doc = nlp2(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('Chinga Tu Madre', 'LOC')]
Tokens [('sin', '', 2), ('luz', '', 2), ('en', '', 2), ('el', '', 2), ('sector', '', 2), ('de', '', 2), ('Chinga', 'LOC', 3), ('Tu', 'LOC', 1), ('Madre', 'LOC', 1), (',', '', 2), ('@Electricaribe', '', 2), ('algún', '', 2), ('daño', '', 2), ('reportado', '', 2), ('?', '', 2), ('gracias', '', 2)]
